This notebook updates the 2.0 crosswalk with known issue fixes, new (ndar subjects) variables, drops free text fields, and merges in latest NDA annotation.  Adds KSADS and SSAGA, too, time allowing. Waiting on NDA for the latter...to finish validation


In [ ]:
import os, datetime
import csv
import sys
import shutil
from openpyxl import load_workbook
import pandas as pd
from io import BytesIO
import numpy as np
import subprocess
from scipy import stats
from ccf.box import LifespanBox
from ccf.config import LoadSettings
from ccf.redcap import RedcapTable 
config = LoadSettings()
from ccf.easy_yaml import EasyYaml
Y = EasyYaml()

In [ ]:
#box=LifespanBox(cache='./cache/')
#verbose = True
snapshotdate = datetime.datetime.today().strftime('%m_%d_%Y')
preppedA="./prepped/hca"
preppedD="./prepped/hcd" 
toolA="/home/petra/NIHToolbox2NDA/prepped/HCA"
toolD="/home/petra/NIHToolbox2NDA/prepped/HCD" 
pathout="."

#Rosetta (a.k.a Inventory) file will have all the nda vars and pedids
#extrainfo=config['rosetta']['filename']

In [ ]:
# get the 2.0 dictionary,drop all nda dd stuff except for element and structure name so that new

#C2=pd.read_csv("./Crosswalk_Lifespan_Behavioral_2.0_01_12_2021.csv")
CD=pd.read_excel("./LS2.0_Crosswalk_Behavioral_Data_Dictionary.xlsx",sheet_name='C-2846 (HCP-D)')
CA=pd.read_excel("./LS2.0_Crosswalk_Behavioral_Data_Dictionary.xlsx",sheet_name='C-2847 (HCP-A)')
print(CD.shape)
print(CA.shape)
C2=pd.concat([CD,CA],axis=0)
print(C2.shape)
C2.collection.value_counts()
C2=C2.drop(columns=['nda_description','nda_notes','nda_range','nda_structure_link'])
C2.head(1)


In [ ]:
#get the header for all prepared structures for list of mapped elements
elems=[]
structures=[]
colls=[]
for preps in [preppedA,preppedD]:# do toolbox stuff later,toolA,toolD]:
    if preps==preppedA:
        collection='C-2847 (HCP-A)'
    if preps==preppedD:
        collection='C-2846 (HCP-D)'
    for filename in os.listdir(preps):
        if 'ssaga' not in filename:
            if 'ksads' not in filename:
                print(filename)
                addem=pd.read_csv(preps+'/'+filename,header=1,nrows=0).columns.tolist()
                l=len(addem)
                m=filename.replace('.csv','')
                if 'eprime' in filename:
                    m='deldisk01'
                structs=[m]*l
                collects=[collection]*l
                elems=elems+addem
                structures=structures+structs
                colls=colls+collects
       

In [ ]:
InPrepped=pd.DataFrame({'nda_element':elems,'nda_structure':structures,'collection':colls})
InPrepped.head(2)
len(InPrepped.nda_structure.unique())

In [ ]:
#toolbox
elems=[]
structures=[]
colls=[]
for preps in [toolA,toolD]:
    if preps==toolA:
        collection='C-2847 (HCP-A)'
    if preps==toolD:
        collection='C-2846 (HCP-D)'
    for filename in os.listdir(preps):
        addem=pd.read_csv(preps+'/'+filename,header=1,nrows=0).columns.tolist()
        #gender/sex aliases all messed up here and at NDA
        addem = [word.replace('gender','sex') for word in addem]
        struc=pd.read_csv(preps+'/'+filename,header=0,nrows=0).columns.tolist()
        m=struc[0]+'01'
        l=len(addem)
        #m=filename.replace('.csv','')
        structs=[m]*l
        collects=[collection]*l
        elems=elems+addem
        structures=structures+structs
        colls=colls+collects
       

In [ ]:
InPreppedT=pd.DataFrame({'nda_element':elems,'nda_structure':structures,'collection':colls})
InPreppedT.head(5)
len(InPreppedT.nda_structure.unique())
' '.join(i for i in list(InPreppedT.nda_structure.unique()))

In [ ]:
#concatenate toolbox with non toolbox
InPrepped=pd.concat([InPrepped,InPreppedT],axis=0)
len(InPrepped.nda_structure.unique())

In [ ]:
#Drop any vars that were excluded in 3.0 (e.g. notes text, etc), by merging the 2.0 
#with the InPrepped
C3init=pd.merge(C2,InPrepped,how='outer',on=['nda_element','nda_structure','collection'],indicator=True)
C3init._merge.value_counts()

#these will need to be found and patched in:
C3init.loc[C3init._merge=='right_only'].to_csv('onlyinprepped.csv',index=False)

#helper files for creating patch:
patchlist=['flanker01','orrt01','prang01','preda01','predd01','psm01','self_effic01','tlbx_emsup01','tlbx_friend01','tlbx_rej01','tlbx_sadness01','tlbx_wellbeing01','tpvt01']
#subset existing crosswalk to structure of interest to facilitate copy/paste
patchtlbx=C3init.loc[(C3init._merge=='right_only') & (C3init.nda_structure.isin(patchlist))]

In [ ]:
#Load the TLBX annotation and reconfigure so it can be used to patch holes for new tlbx elements
crosswalkpath="/home/petra/NIHToolbox2NDA/"
cfile="Crosswalk_NIH_Toolbox_2_NDA.csv"
crosswalk=pd.read_csv(crosswalkpath+cfile,header=0,low_memory=False, encoding = "ISO-8859-1")
crosswalk=crosswalk.drop(columns=['Measurement System','validated','DataType','specialty_code','template','inst_short','Source','description','valueRange','notes'])

crosswalk=crosswalk.loc[crosswalk.nda_structure.isin(patchlist)]
crosswalk=crosswalk.rename(columns={'Responses':'hcp_choices_calcs','Inst':'hcp_instrument','hcp_variable':'hcp_variable_name',
                          'action_requested':'nda_request','requested_python':'trans_func','description':'nda_description',
                                    'valueRange':'nda_range','notes':'nda_notes','Source':'hcp_source'})

crosswalk['hcp_label']=crosswalk['Domain'].fillna('') +":"+ crosswalk['Item ID'].fillna('') +":"+ crosswalk['Stem'].fillna('') +":"+ crosswalk['Context'].fillna('')
crosswalk['hcp_label']=crosswalk['hcp_label'].str.strip(":")
crosswalk.loc[crosswalk.hcp_label.fillna('')=='','hcp_label']=crosswalk.hcp_variable_name 
crosswalk=crosswalk.drop(columns=['Domain','Item ID','Stem','Context'])
                                  
crosswalk['source']='NIH Toolbox'
crosswalk=crosswalk.loc[crosswalk.nda_element.isnull()==False]
crosswalk.columns
crosswalk['hcp_description']=''
crosswalk['trans_nda_request']=''
crosswalk['hcp_source']=''

#another helper file for creating patch:
toolboxpatch=pd.merge(patchtlbx[['collection','nda_element','nda_structure']],
        crosswalk[['nda_element','nda_structure','hcp_description','hcp_source','hcp_variable_name','hcp_label','hcp_instrument','hcp_choices_calcs',
           'trans_nda_request','trans_func']],on=['nda_element','nda_structure'],how='left')
toolboxpatch.to_csv('patchtest.csv',index=False)

In [ ]:
#load patch (created offline from list of elements in prepped only and fleshed out with stuff from tlbx, etc)
C3=pd.merge(C2,InPrepped,how='inner',on=['nda_element','nda_structure','collection'])
patch=pd.read_csv("./CrosswalkPatch.csv")
print(C3.shape)
patch.shape
print(C3.columns)
print(patch.columns)

C4=pd.concat([C3,patch],axis=0)

#add in the in-scanner facename variables, even though they aren't ready for 3.0 distro
C4=C4.loc[~(C4.nda_structure=='facename01')]
face=C2.loc[C2.nda_structure=='facename01']
C4new=pd.concat([C4,face],axis=0)

#cleanup
C4=C4new.drop(columns=['nda_aliases','nda_description','nda_notes','nda_range','nda_structure_link'])

#double check that C4 has everything needed in terms of maps 
C4init=pd.merge(C4,InPrepped,how='outer',on=['nda_element','nda_structure','collection'],indicator=True)
C4init.loc[C4init._merge=='right_only'].to_csv('onlyinprepped.csv',index=False)



In [ ]:
#add ssaga and ksads...use the NDA data dictionary for these structures
ssaga1=pd.read_csv(preppedA+'/ssaga_part101.csv',header=1,nrows=0).columns.tolist()
Ssaga1=pd.DataFrame(ssaga1)
Ssaga1['nda_structure']='ssaga_part101'

ssaga2=pd.read_csv(preppedA+'/ssaga_part201.csv',header=1,nrows=0).columns.tolist()
Ssaga2=pd.DataFrame(ssaga2)
Ssaga2['nda_structure']='ssaga_part201'

Ssaga=pd.concat([Ssaga1,Ssaga2],axis=0)
Ssaga['collection']='C-2847 (HCP-A)'
Ssaga.columns=['nda_element','nda_structure','collection']

Ssaga.head()
Ssaga.nda_structure.value_counts()


In [ ]:
ksads=pd.read_csv(preppedD+'/hcpksads01.csv',header=1,nrows=0).columns.tolist()
Ksads=pd.DataFrame(ksads)
Ksads['nda_structure']='hcpksads01'
Ksads['collection']='C-2847 (HCP-D)'
Ksads.columns=['nda_element','nda_structure','collection']
Ksads.head()

In [ ]:
#Put it all together
C5=pd.concat([C4,Ssaga,Ksads])
len(C5.nda_structure.unique())

In [ ]:
# make sure all the rosetta vars look good.
C5.loc[C5.hcp_source=='rosetta','hcp_instrument']='Inventory'
# add SSAGA and KSADS, and double check that all TLBX vars are present, since some
# might not have made it to first crosswalk due to smaller sample


In [ ]:
#merge fresh NDA information and then flesh out Ksads and SSaga 'hcp_source','hcp_description' and 'values'.
#drop harmonized harmonization_index
C5.head()


In [ ]:
#get the NDA maps
nda = {}
for filename in os.listdir('./nda/'):
    struct = filename[:-5]
    elements = Y('./nda/'+filename)
    nda[struct] = elements

In [ ]:
structs=[]
for i in nda.keys():
    structs=structs+[i]

type(structs)
len(structs)
#structs

In [ ]:
structs=[]
for i in nda.keys():
    structs=structs+[i]
    c=pd.DataFrame()

for j in structs:
    #print(j)
    a=pd.DataFrame.from_dict(nda[j])
    b=a.transpose()
    b=b.reset_index()
    b['nda_structure']=j
    c=pd.concat([c,b])
    

c.nda_structure.unique()


In [ ]:
c['index']

In [ ]:
d=c.rename(columns={'index':'nda_element','description':'nda_description','range':'nda_range',
                  'notes':'nda_notes','alias':'nda_aliases'})
d['nda_structure_link']='https://nda.nih.gov/data_structure.html?short_name='+c['nda_structure']
d=d.drop(columns=['type','required','length','codes','condition'])
d.columns
d=d[['nda_element','nda_description', 'nda_notes','nda_range', 'nda_aliases', 
       'nda_structure']]



In [ ]:
print(d.nda_structure.unique())
d.nda_element.head(5)

In [ ]:
d.head(2)
d.nda_element
d.nda_structure
d.loc[d.nda_structure=='ipaq01'].head()
C5.loc[C5.nda_structure=='ipaq01'].head()
C5.nda_structure.unique()


In [ ]:
crosswalk=pd.merge(C5,d,how='left',on=['nda_structure','nda_element'])
crosswalk['nda_structure_link']='https://nda.nih.gov/data_structure.html?short_name='+crosswalk['nda_structure']
crosswalk.nda_structure.unique()

In [ ]:
#crosswalk.loc[crosswalk.collection.str.contains('([sup_1_y]:[sup_7_y])')]
#crosswalk.loc[crosswalk.collection.str.contains('; Validation: All items must be answered')]
#C-2846 (HCP-D)	sup_y_ss_sum	gbi01	7UP sum of all Validation: All items must be answered	teen/child	s7upchildself_sum	7UP sum of all Validation: All items must be answered	up_scores:The 7 Up Inventory	if([s7upchildself_nm]>=1, "NaN", sum([s7upchildself1], [s7upchildself2], [s7upchildself3], [s7upchildself4], [s7upchildself5], [s7upchildself6], [s7upchildself7]))					STRUCTURE MISMATCH	102346		7UP sum
#([sup_1_y]:[sup_7_y])																
#; Validation: All items must be answered			s7upchildself_sum	https://nda.nih.gov/data_structure.html?short_name=gbi01												
#crosswalk.loc[crosswalk.collection=='C-2846 (HCP-D)'].shape

In [ ]:
crosswalk.to_csv('Crosswalk_Lifespan_3.0.csv',index=False)